In [ ]:
def percent_change(percent,date_of_event, value_compare_to, direction, data_supplied ):
    
    
    XRP_Data = data_supplied
    
    "next time it was increased by {0}%".format(percent)
    #print(date_of_event)
    if direction.lower() == 'increase':
        change = value_compare_to* (1+ percent*0.01)
        
        if len(XRP_Data[(XRP_Data['date'] > date_of_event) & (XRP_Data['high'] >= change) ]) == 0:
            return 3456789123
        else:
            return XRP_Data[(XRP_Data['date'] > date_of_event) & (XRP_Data['high'] >= change) ].iloc[[0]]['date'].values[0]
    
    else:
        change = value_compare_to* (1- percent*0.01)
        
        if len(XRP_Data[(XRP_Data['date'] > date_of_event) & (XRP_Data['low'] <= change) ]) == 0:
            return 3456789123
        else:
            return XRP_Data[(XRP_Data['date'] > date_of_event) & (XRP_Data['low'] <= change) ].iloc[[0]]['date'].values[0]

In [ ]:
time_event = {}
starting_asset = 100
bought_crypto = False


gold_available = starting_asset 
XRP_amount = 0


drop_limit = 0.005 # used mean on this side, actual 0.004686, but decreased abit for optimistic view 

stop_loss_limit = 0.01
sell_limit = 0.004

while gold_available >= 0:
    
    for ind, df in XRP_Analysis_table.iterrows():
#         print(df['date'])
        
        if bought_crypto != True: # then looking to buy at drop limit or lower
            if ( df['2H_RA_on_Close'] - df['low'] ) / df['2H_RA_on_Close'] >= drop_limit:
                "then buy crypto"

                XRP_amount += (gold_available/df['low']) * 0.998 # due to 0.2% fee

                gold_available -= gold_available
                
                time_event.update({df['date']  : ( gold_available, XRP_amount , 'brought' , df['low'])})
                
                last_xrp_traction_price =  df['low']
                
                bought_crypto = True
                
#                 print('last_xrp_traction_price ', last_xrp_traction_price)
                
        else:
            if bought_crypto: # has crypto, either wait or sell at stoploss or profit
#                 print('hit_sell_limit',
#                       (df['high'] - last_xrp_traction_price)/last_xrp_traction_price >= sell_limit ,
#                       round((df['high'] - last_xrp_traction_price),6) ,
#                       'sell_limit ', sell_limit
                     
#                       )
                
                if ( df['2H_RA_on_Close'] - df['low'] ) / df['2H_RA_on_Close'] >= stop_loss_limit: # then hit stop loss, force sell
#                     print('stoppppppppppppppppppppppppp')
                    gold_available += XRP_amount * df['low'] * 0.998  # sell at low price
                    
                    XRP_amount -= XRP_amount
                    
                    time_event.update({df['date']  : ( gold_available, XRP_amount , 'stop_loss_hit' ,  df['low'])})
                    
                    bought_crypto = False
                    
                    
                
                elif  (df['high'] - last_xrp_traction_price)/last_xrp_traction_price >= sell_limit: # then hit target price, sell for profit
                    
#                     print(last_xrp_traction_price)
                                        
                    gold_available += XRP_amount * df['high'] * 0.998  # sell at high price
                    
                    XRP_amount -= XRP_amount
                    
                    time_event.update({df['date']  : ( gold_available, XRP_amount , 'sell_profit' , df['high'])})
                    
                    bought_crypto = False

    break
                
else:
    print('Negative asset: bankrupt  ->>>   ',  gold_available)